NameError: name 'requests_toolbelt' is not defined

Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests
# import curlify
import json
import os
import pydicom as dicom
import time
import base64
import glob
import datetime
import names
import random
from faker import Faker
import tempfile
from requests_toolbelt.multipart.encoder import MultipartEncoder
from glob import glob
from random import randrange
def series_web_upload(filepaths, server_base_url, token):
    url = "{}/studies/)".format(server_base_url)
    print('uploading dicom to {} with token {}'.format(url, token))
    m = MultipartEncoder({
            file_path: (os.path.basename(file_path), open(file_path, 'rb'), 'application/dicom')
            for file_path in filepaths
        })
    file_count = len(filepaths)
    print(f'number of files uploading: {file_count}')
    content_type = 'multipart/form-data; type=application/dicom; boundary={}'.format(m.boundary_value)
    print(m.boundary_value)
    # Do the HTTP POST request to the STOW-RS server
    # print(f'lenght {m._calculate_length()}')
    r = requests.post(
        url=url,
        timeout=60,
        data=m,
        headers={
            'Content-Type': content_type,
            'Authorization': 'Token {}'.format(token),
            'Accept': 'application/json',
            'source': "qremote_dev",
            'user': 'qremote',
            'NFILES': str(file_count)
        })
    return r


def make_compute_call(series_instance_uid, server_base_url, token, force=False):
    url = "{}/compute/".format(server_base_url)
    print('initiating trigger for {} with token {}'.format(url, token))
    start = time.time()
    compute_endpoint = '{}{}'.format(url, series_instance_uid)
    print(compute_endpoint)
    if force:
        compute_endpoint = compute_endpoint + '?recompute=True'
    r = requests.get(compute_endpoint,
    headers={'Authorization': 'Token {}'.format(token)})
    end = time.time()
    print(r.content)
    print("Took {} seconds".format(end-start))
    return r


def get_results(series_instance_uid, server_base_url, token):
    url = "{}/results/".format(server_base_url)

    print('requesting results for {} with token {}'.format(url, token))
#     start = time.time()
    r = requests.get(
        '{}{}'.format(url, series_instance_uid),
        headers={'Authorization': 'Token {}'.format(token), 'source':'webportal'})
#     end = time.time()
    # print(r.json())
#     print("Took {} seconds".format(end-start))
    return r

def increment_identifier(identifier):
    identifier_split = identifier.split(".")
    suffix = int(identifier_split[-1])
    suffix = suffix + 190
    identifier_split[-1] = str(suffix)
    identifier_incremented = ".".join(identifier_split)
    return identifier_incremented

def update_identifiers(filepath, patient_metadata):
    try:
        dcm_obj = dicom.read_file(filepath)
        sopInstanceUID = str(dcm_obj.SOPInstanceUID)
        seriesInstanceUID = str(dcm_obj.SeriesInstanceUID)
        studyInstanceUID = str(dcm_obj.StudyInstanceUID)

        dcm_obj.SOPInstanceUID = increment_identifier(sopInstanceUID)
        
        dcm_obj.SeriesInstanceUID = increment_identifier(seriesInstanceUID)

        dcm_obj.StudyInstanceUID = increment_identifier(studyInstanceUID)

        if patient_metadata:
            dcm_obj.PatientName = patient_metadata.get('name', 'Scooby Doo')
            dcm_obj.PatientBirthDate = patient_metadata.get('dob', datetime.date.today())
            dcm_obj.PatientSex = patient_metadata.get('gender', 'M')
            dcm_obj.PatientID = patient_metadata.get('id')

        dicom.write_file(filepath, dcm_obj)
        return filepath
    except Exception as e:
        return None
    
def chunks(lst, n):
    """Yield tuple of (chunk, is_last_chunk) from lst."""
    for i in range(0, len(lst), n):
        yield lst[i: i + n], int(i/n)

In [22]:
my_token={
    'qremote_demo': 'W5eE1g8JIql5yrfmFN3LqXvrzBeQEeXe',
    'vrad': 'onpremise',
    'onpremise': 'l44ih5phgxv3kmio7oiprnbezethtd2vr9nfkvhb',
}

server_base_url={
    "prod_eu":"https://qremoteapieu.qure.ai",
    "prod_us":"https://platformapius.qure.ai",
    "preprod":"https://platformapi-preprod.qure.ai",
    "platform": "https://platformapi.qure.ai",
    "staging": "https://platformapi-staging.qure.ai",
    "sprint": "https://platformapi-sprint.qure.ai",
    "rnd": "https://platformapi-rnd.qure.ai",
    "dev3": "https://e53e-43-204-126-249.in.ngrok.io",
    "demo_app": "https://demo-appapi.qure.ai",
    "local": "http://0.0.0.0:2005",
    "qbox": "http://0.0.0.0:59257",
    
}

In [23]:
def get_file_list(directory):
    files_list = glob(directory+"/**", recursive = True)
    list_dcm=[f for f in sorted(files_list)]
    print(len(files_list),len(list_dcm))
    return sorted(list_dcm)

In [24]:
def generate_patient_data():
    fake = Faker()
    genders = ['M', 'F']
    patient_metadata = {
        'name': fake.name(),
        'gender': random.choice(genders),
        'id': fake.pystr_format(),
        'dob': fake.date_between(start_date='-70y', end_date='today')
    }

    print(patient_metadata)
    # patient_metadata['PATIENT_ID'] = 'x9-487991c'
# print(patient_metadata)
    return patient_metadata

In [25]:
def upload_files_fun(list_dcm, patient_metadata):
    upload_files = list()
    
    for filepath in list_dcm:

        filepath = update_identifiers(filepath, patient_metadata)
        if filepath:
            upload_files.append(filepath)
            
    return upload_files

In [26]:
my_server=server_base_url['local']
token=my_token['onpremise']

In [27]:
def push_files(upload_files):
    for chunk, batch in chunks(upload_files, 150):
        a=series_web_upload(chunk, my_server, token)
        print(a)
    return a
def series_compute(series_instance_uids):
    r=make_compute_call(series_instance_uids, my_server, token, True)

In [41]:
# push multiple studies
for i in range(1):
    series_list=[]
    directory = "/Users/paras/work/studies/qxr_f/**"
    study_list = glob(directory, recursive = True)
    p_data = generate_patient_data()
    upload_files=upload_files_fun(study_list, p_data)
    r=push_files(upload_files)
    # series_list.append(r.json().get('series_instance_uids'))
    # for s in series_list:
    #     if s:
    #         series_compute(s[0])
#     time.sleep(10)

{'name': 'Becky Morgan', 'gender': 'F', 'id': 'k5-5774211Z', 'dob': datetime.date(2020, 1, 4)}
uploading dicom to http://0.0.0.0:2005/studies/) with token l44ih5phgxv3kmio7oiprnbezethtd2vr9nfkvhb
number of files uploading: 2
7ff5a54c756a416bbc4664816e5fd36c


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [255]:
# print(r.json())

In [176]:
series_list=[['1.2.826.0.1.3680043.8.498.44646960912168799191637963173975651303']]

In [177]:
print(series_list)
for s in series_list:
    series_compute(s[0])

[['1.2.826.0.1.3680043.8.498.44646960912168799191637963173975651303']]
initiating trigger for http://0.0.0.0:2005/compute/ with token onpremise
http://0.0.0.0:2005/compute/1.2.826.0.1.3680043.8.498.44646960912168799191637963173975651303
b'{"message":"1.2.826.0.1.3680043.8.498.44646960912168799191637963173975651303 successfully pushed for re-computation!","success":true}'
Took 0.09781002998352051 seconds


In [156]:
r=get_results('1.3.6.1.4.1.5962.99.1.37111204590.849306716.1543.0.31107', my_server, token)
print(r.json())

requesting results for http://0.0.0.0:2005/results/ with token onpremise
{'success': True, 'message': 'Results retrieved successfully!', 'triage_status': 'Routine', 'result': {'report': {'findings': 'Fibrosis observed in bilateral upper and lower zones\n\nNo blunting in CP angles is seen.\n\nNo significant findings observed in the heart.\n\nMediastinum is within normal limits.\nTrachea is centrally located.\n\n', 'impression': 'Abnormal study', 'findings_list': [['bone', 'Bony thorax appears unremarkable.\n\n'], ['lung', 'Fibrosis observed in bilateral upper and lower zones\n\n'], ['heart', 'No significant findings observed in the heart.\n\n'], ['hilum', 'Both hila appear normal.\n\n'], ['pleura', 'No blunting in CP angles is seen.\n\n'], ['mediastinum', 'Mediastinum is within normal limits.\nTrachea is centrally located.\n\n']]}, 'files': {'sc': ['/srv/data/hct/onpremise/1.3.6.1.4.1.5962.99.1.37111204590.849306716.1543.0.31107/27821523-2ca1283c-87cae67c-c1123961-0f2d4c7f_sc.dcm'], 'gs

In [184]:
["Fibrosis observed in bilateral upper and lower zones","No blunting in CP angles is seen."] \n\nNo significant findings observed in the heart.\n\nMediastinum is within normal limits.\nTrachea is centrally located.\n\n

In [ ]:
Fibrosis observed in bilateral upper and lower zones\n\nNo blunting in CP angles is seen.\n\nNo significant findings observed in the heart.\n\nMediastinum is within normal limits.\nTrachea is centrally located.\n\n